# Import Necessary Modules

In [ ]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

## Loading Webdriver for selenium

In [ ]:
ser = Service('/home/irfan/Downloads/chromedriver_linux64')

## Initializing web driver

In [ ]:
myoptions = Options()
driver = Chrome(service=ser, options=myoptions)

## Opening the browser or redirecting to the link using get method

In [ ]:
driver.get('https://www.eduvision.edu.pk/')

### Using Selenium Selectors, find all the anchor tags having class = 'list_hlaf_block' <br> and extract Universities name nested in anchors tags. <br>Scrape the home page and got anchor tags of all sectors 

In [ ]:
sectors_name = []
sectors_links = []
anchors = driver.find_elements(By.CLASS_NAME, "list_hlaf_block")
anchors = anchors[-5:-7:-1]
for a in anchors:
    b = a.find_element(By.CSS_SELECTOR, 'b')
    sectors_name.append(b.text)
for a in anchors:
    sectors_links.append(a.get_attribute('href'))
print(sectors_links)
print(sectors_name)

### Scraping the links and names of universities of all Sectors

In [ ]:
universities = {}
for link in sectors_links:
    driver.get(link)
    all_divs = driver.find_elements(By.CLASS_NAME, "threecolumn")
    for div in all_divs:
        d = div.find_element(By.CLASS_NAME, 'fixText')
        a = d.find_element(By.CSS_SELECTOR, 'a')
        anchor = a.get_attribute('href')
        name = a.text
        universities[name] = anchor
universities

### Scraping the Indvidual University Informations

In [ ]:
university_degrees = []
degree_programs = {}
tmp_stored_programs_levels = []
for name, link in universities.items():
    driver.get(link)
    #Label = driver.find_element(By.CLASS_NAME, 'tabs')
    #inputs = Label.find_elements(By.CSS_SELECTOR, 'input')
    #tabs = Label.find_elements(By.CSS_SELECTOR, 'label')
    #for i in range(len(tabs)):
        #txt = tabs[i].text
        #dict = {txt : None}
        #degree_programs.append(dict)
        #tmp_stored_programs_levels.append(txt)
    label = driver.find_elements(By.CLASS_NAME, 'label')
    tbody = driver.find_elements(By.CSS_SELECTOR, 'tbody')
    i = 0
    for body in tbody:
        tr = body.find_elements(By.CSS_SELECTOR, 'tr')
        grade_level_data = {}
        for row in tr:
            td = row.find_elements(By.CSS_SELECTOR, 'td')
            indvidual_program_info = []
            for data in td:
                indvidual_program_info.append(data.text)
            grade_level_data[indvidual_program_info[0]] = indvidual_program_info[1:]
        if len(label) > 0:
            degree_programs[label[i].text] = grade_level_data
            i += 1
        else:
            degree_programs['Programs'] = grade_level_data
        print(degree_programs)
#     panel = Label.find_elements(By.CLASS_NAME, 'panel')
#     j = 0
#     for t in panel:
#         table = t.find_element(By.CSS_SELECTOR, 'table')
#         tbody = table.find_element(By.CLASS_NAME, 'para')
#         tr = tbody.find_elements(By.CSS_SELECTOR, 'tr')
#         programs_info = {}
#         for row in tr:
#             td = row.find_elements(By.CSS_SELECTOR, 'td')
#             name = td[0].find_element(By.CSS_SELECTOR, 'a').text
#             duration = td[1].text
#             fee = td[2].text
#             #deadline = td[3].find_element(By.CSS_SELECTOR, 'a').text
#             deadline = td[3].text
#             merit = td[4].text
#             programs_info[name] = [duration, fee, deadline, merit]
#         degree_programs[j][tmp_stored_programs_levels[j]] = programs_info
#         j += 1
#         university_degrees.append(degree_programs)
# print(university_degrees)